In [2]:
import wmfdata as wmf

This..doesn't actually work properly at the moment. A couple issues:
* The country name "Bonaire, Sint Eustatius, and Saba" needs to be escaped because it contains commas. By default, Pandas does this by wrapping the field in double quotes, but that causes silent breakage in the loaded Hive table in that row. There's probably a way to fix this by altering the escaping settings, but I worked around it by saving it as a Parquet file and then using Spark to load the Parquet file directly to Hive. Since Parquet is a better format than CSV, it doesn't have the escaping problems.
* Currently, the `analytics-product` user owns the `canonical_data` database. This means that, to operate on it, you have to have rights to sudo as that user and then you have to run the command as `sudo -u analytics-product kerberos-run-command analytics-product hive -e "{{SQL}}"`. This can't be done from inside our Jupyter environment. I worked around this by doing the actual loading into a table in my own database inside Jupyter, then in a plain SSH connection doing the sudo stuff to run the queries `DROP TABLE canonical_data.countries` and `CREATE TABLE canonical_data.countries AS SELECT * FROM nshahquinn.countries`.
* (Bonus note: If you load the data from a CSV into Pandas, make sure Namibia's country code "NA" is not intepreted as a null.)

In [8]:
wmf.hive.load_csv(
    "countries.csv",
    field_spec="""
        name STRING,
        iso_code STRING,
        economic_region STRING,
        maxmind_continent STRING,
        is_protected BOOLEAN
    """,
    db_name="nshahquinn",
    table_name="countries"
)

/home/neilpquinn-wmf/.conda/envs/2022-11-30T20.59.36_neilpquinn-wmf/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
